In [4]:
!pip install delu rtdl_revisiting_models

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.6/41.6 kB 1.5 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl (121.6 MB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl (56.5 MB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl (124.2 MB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manylinux1_x86_64.whl (196.0 MB)
  Using cached nvidia_nccl_cu12-2.20.5-py3-none-manylinux2014_x86_64.whl (176.2 MB)
  Using cached nvidia_nvtx_cu12-12.1.105-py3-none-man

In [5]:
import pandas as pd
import numpy as np
import math
from sklearn.model_selection import train_test_split

from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import StandardScaler

pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)

import delu
#import rtdl
from rtdl_revisiting_models import FTTransformer

from tqdm.std import tqdm

import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import TensorDataset, DataLoader, Dataset
from functools import partial
import sklearn.model_selection
import sklearn.preprocessing
from sklearn.metrics import recall_score

In [6]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [7]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [28]:
name = 'fraud'# which type of dataset you need? Options are: fraud or default

In [29]:
if name == 'fraud':
  df = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/fraud_dataset.csv')
  df = df.rename(columns={'Loan Status': 'target'})
  df.head()
elif name == 'default':
  df = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/default_dataset.csv')
  df.head()
else:
  print('Smth is wrong')

In [30]:
y = df[['target']]
X = df.drop(columns=['target'])

### preparing dataset for Transformer model:

In [31]:
X_cat = X.select_dtypes(include=['object'])

X_cat = X_cat.apply(LabelEncoder().fit_transform)

X_float = X.select_dtypes(include=['float'])
X_int_float = X.select_dtypes(include=['int64']).astype(np.float32)
X_float = pd.concat([X_float, X_int_float], axis=1)
X_float = X_float.astype(np.float32)

float_features_cnt = X_float.shape[1]
cat_features_card = [X_cat[x].nunique() for x in X_cat.columns]

y = y.astype('float')

In [32]:
X_float = X_float.to_numpy()
X_cat = X_cat.to_numpy()
y = y.to_numpy()
y = y[:, 0]

In [33]:
idx = np.arange(len(y))
train_val_idx, test_idx = sklearn.model_selection.train_test_split(
    idx, train_size=0.8, random_state=1
)
train_idx, val_idx = sklearn.model_selection.train_test_split(
    train_val_idx, train_size=0.8, random_state=1
)

In [34]:
data = {
    'train' : {'x_float':X_float[train_idx], 'x_cat':X_cat[train_idx], 'y':y[train_idx]},
    'val' : {'x_float':X_float[val_idx], 'x_cat':X_cat[val_idx], 'y':y[val_idx]},
    'test' : {'x_float':X_float[test_idx], 'x_cat':X_cat[test_idx], 'y':y[test_idx]}
}

In [35]:
preprocessing = sklearn.preprocessing.StandardScaler().fit(data['train']['x_float'])

for object in data:
    data[object]['x_float'] = preprocessing.transform(data[object]['x_float'])

In [36]:
tensors = {
    object: {k: torch.as_tensor(v, device=device) for k, v in data[object].items()} for object in data
}

In [37]:
tensors

{'train': {'x_float': tensor([[ 0.3307, -1.1457, -0.8003,  ..., -0.1797, -0.6375,  0.7899],
          [ 1.7731,  0.2373, -0.3878,  ..., -0.1204, -0.5689, -0.5167],
          [-0.5876, -0.4282, -0.6920,  ..., -0.1362, -0.8748,  0.9690],
          ...,
          [ 0.1390,  0.6916, -1.3650,  ..., -0.1599, -0.9967,  4.0163],
          [-0.5288, -1.7436,  0.6120,  ..., -0.1467,  2.0716,  0.3716],
          [-0.2501,  0.8358, -0.1778,  ..., -0.1138, -0.9284,  0.7432]],
         device='cuda:0'),
  'x_cat': tensor([[ 1,  8,  0,  1,  1,  1],
          [ 2, 11,  0,  1,  0,  0],
          [ 3,  5,  0,  1,  0,  0],
          ...,
          [ 5,  7,  1,  0,  1,  1],
          [ 4,  3,  2,  1,  1,  0],
          [ 4, 13,  1,  1,  1,  1]], device='cuda:0'),
  'y': tensor([0., 0., 0.,  ..., 0., 1., 0.], device='cuda:0', dtype=torch.float64)},
 'val': {'x_float': tensor([[-1.3033,  1.2924,  0.1889,  ..., -0.1375, -0.8578,  0.2755],
          [-0.8993,  0.7421,  0.7407,  ..., -0.1204, -0.6062, -1.0525]

### Transformer model

In [55]:
model = FTTransformer(
    n_cont_features=float_features_cnt,
    cat_cardinalities=cat_features_card,
    d_out=1,
    **FTTransformer.get_default_kwargs(),
).to(device)
optimizer = model.make_default_optimizer()

In [56]:
def apply_model(batch):
    return model(batch['x_float'], batch['x_cat']).squeeze(-1)

In [57]:
pos_weight = 10 if name == 'fraud' else 4
loss_func = nn.BCEWithLogitsLoss(pos_weight = torch.tensor(pos_weight))

In [58]:
def evaluate(part, eval_batch_size):
    model.eval()
    with torch.inference_mode():
        y_pred = (
            torch.cat(
                [
                    apply_model(batch) for batch in delu.iter_batches(tensors[part], eval_batch_size)
                ]
            )
            .cpu()
        )
        y_true = tensors[part]['y'].cpu().numpy()

        y_pred = torch.sigmoid(y_pred).numpy()
        test_y_pred = y_pred
        y_pred = np.round(y_pred)
        score = sklearn.metrics.accuracy_score(y_true, y_pred)
        recall = sklearn.metrics.recall_score(y_true, y_pred)
    return score, recall, test_y_pred

In [59]:
n_epochs = 100
patience = 100

eval_batch_size = 256
batch_size = 256
epoch_size = math.ceil(len(train_idx) / batch_size)
timer = delu.tools.Timer()
early_stopping = delu.tools.EarlyStopping(patience, mode='max')

print(f"Device: {device}")
print("-" * 88 + "\n")
timer.run()
for epoch in range(n_epochs):
    for batch in tqdm(
        delu.iter_batches(tensors['train'], batch_size, shuffle=True),
        desc=f"Epoch {epoch}",
        total=epoch_size,
    ):
        model.train()
        optimizer.zero_grad()
        loss = loss_func(apply_model(batch), batch['y'])
        loss.backward()
        optimizer.step()

    val_score, val_recall, _ = evaluate('val', eval_batch_size)
    print(f"Accuracy: (val) {val_score:.4f} [time] {timer}")
    print(f"Recall: (val) {val_recall:.4f} [time] {timer}")

    early_stopping.update(val_score)
    if early_stopping.should_stop():
        break

    print()

Device: cuda
----------------------------------------------------------------------------------------



Epoch 0: 100%|██████████| 169/169 [00:03<00:00, 45.71it/s]


Accuracy: (val) 0.0932 [time] 0:00:04.074754
Recall: (val) 1.0000 [time] 0:00:04.075672



Epoch 1: 100%|██████████| 169/169 [00:03<00:00, 46.43it/s]


Accuracy: (val) 0.7018 [time] 0:00:08.091977
Recall: (val) 0.2971 [time] 0:00:08.092069



...

Accuracy: (val) 0.7864 [time] 0:07:02.095056
Recall: (val) 0.1406 [time] 0:07:02.095163



Epoch 98: 100%|██████████| 169/169 [00:03<00:00, 42.75it/s]


Accuracy: (val) 0.7645 [time] 0:07:06.459287
Recall: (val) 0.1665 [time] 0:07:06.459408



Epoch 99: 100%|██████████| 169/169 [00:03<00:00, 42.70it/s]


Accuracy: (val) 0.7710 [time] 0:07:10.827846
Recall: (val) 0.1595 [time] 0:07:10.827964



In [60]:
test_score, test_recall, prob= evaluate('test', eval_batch_size)
print(f"Accuracy: (test) {test_score:.4f}")
print(f"Recall: (test) {test_recall:.4f}")
print(prob[tensors['test']['y'].cpu()==1])
print(tensors['test']['y'][(tensors['test']['y']==1)])

Accuracy: (test) 0.7720
Recall: (test) 0.1657
[0.10362696 0.00135738 0.00141008 ... 0.00073575 0.03481104 0.00435947]
tensor([1., 1., 1.,  ..., 1., 1., 1.], device='cuda:0', dtype=torch.float64)


In [63]:
sklearn.metrics.confusion_matrix(tensors['test']['y'].cpu().numpy(), np.round(prob))

array([[10190,  2029],
       [ 1042,   207]])